In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-san

In [ ]:
# KoNLPy 설치
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

import torchtext

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

from konlpy.utils import pprint
from konlpy.tag import Okt

from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression

import pyLDAvis.sklearn

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


# 웹 서버 요청

In [ ]:
def web_server_request(code_number):
    # 네이버 영화 사이트 - 영화 리뷰 페이지
    url = "https://movie.naver.com/movie/bi/mi/review.naver?code={}".format(code_number)

    # reqeust.get
    resp = requests.get(url)
    return resp

In [ ]:
doctor = web_server_request(182016)
thor = web_server_request(187347)
black = web_server_request(184516)

In [ ]:
print(doctor)
print(thor)
print(black)

<Response [200]>
<Response [200]>
<Response [200]>


In [ ]:
doctor.text[150:350]

'ng="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="imagetoolbar" content="no">\r\n<title>닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화</title>\r\n\r\n\t\r\n\t\r\n\t\r\n'

In [ ]:
thor.text[150:350]

'ng="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="imagetoolbar" content="no">\r\n<title>토르: 러브 앤 썬더 : 네이버 영화</title>\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t'

In [ ]:
black.text[150:350]

'ng="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="imagetoolbar" content="no">\r\n<title>블랙 팬서: 와칸다 포에버 : 네이버 영화</title>\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r'

# HTML 파싱

In [ ]:
# 3개의 영화 서버 응답 텍스트 리스트 생성
movie_text = [doctor.text, thor.text, black.text]

# 파싱한 내용을 담을 리스트 생성
soup_list = []

# BeautifulSoup 함수로, HTML 문서 구조를 파싱
for text in movie_text:
    soup = BeautifulSoup(text, 'html.parser')
    soup_list.append(soup)
    # 파싱한 내용을 담고 있는 객체의 자료형 확인
    print(type(soup))

<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>


## 영화 제목

In [ ]:
for soup in soup_list:
    # <title> 영화 이름 : 네이버 영화</title>
    # title 태그 이름을 활용하여 영화 제목이 포함되어 있는 요소를 찾습니다.
    title_tag = soup.find("title")
    print(title_tag)

    # 텍스트 부분만 출력
    title_text = title_tag.get_text()
    print(title_text)

<title>닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화</title>
닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화
<title>토르: 러브 앤 썬더 : 네이버 영화</title>
토르: 러브 앤 썬더 : 네이버 영화
<title>블랙 팬서: 와칸다 포에버 : 네이버 영화</title>
블랙 팬서: 와칸다 포에버 : 네이버 영화


## 리뷰 갯수

In [ ]:
for soup in soup_list:
    # <span class="cnt">총<em>건수</em>건</span>

    # span 태그의 class 속성값을 활용하여 리뷰 갯수가 포함되어 있는 요소를 찾습니다.
    count_tag = soup.find("span", attrs = {'class': 'cnt'})
    print("span 태그: ", count_tag)

    # count_tag 요소에서 em 태그 부분을 찾습니다.
    count_tag = count_tag.find('em')
    print("em 태그: ", count_tag)

    # 텍스트 부분만 추출합니다.
    count_text = count_tag.get_text()
    print("텍스트: ", count_text)

span 태그:  <span class="cnt">총<em>140</em>건</span>
em 태그:  <em>140</em>
텍스트:  140
span 태그:  <span class="cnt">총<em>67</em>건</span>
em 태그:  <em>67</em>
텍스트:  67
span 태그:  <span class="cnt">총<em>19</em>건</span>
em 태그:  <em>19</em>
텍스트:  19


## 리뷰 목록

In [ ]:
code_number = [182016, 187347, 184516] # 각 영화별 페이지 code number
i = 0 # 초기 값 설정

# 30개의 영화 제목, 리뷰 제목, 사용자, url을 담을 리스트 생성
movie_list = []
title_list = []
uid_list = []
url_list = []

for soup in soup_list:
    # <ul class="rvw_list_area">

    # ul 태그의 class 속성값을 활용하여 리뷰 제목과 링크가 포함되어 있는 요소를 찾습니다.
    review_list_tag = soup.find('ul', attrs = {'class':'rvw_list_area'})

    # review_list_tag 요소에 포함된 li 태그를 모두 찾습니다.
    review_list_tags = review_list_tag.find_all('li')

    # 10개의 리뷰를 반복문으로 조회하면서, 영화제목, 리뷰 제목(rli.title), 사용자(rli.uid), 상세 페이지 url 값을 추출합니다.
    for li_tag in review_list_tags:
        title_text = soup.find("title").get_text()
        movie_list.append(title_text)

        review_title = li_tag.find_all('a')[0].get_text()
        title_list.append(review_title)

        review_uid = li_tag.find_all('a')[1].get_text()
        uid_list.append(review_uid)

        review_nid = re.findall('\d{7}', li_tag.find('a').get('onclick'))[0]
        review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code={code_number[i]}&order=#tab"
        url_list.append(review_url)
    i += 1 # 영화 리뷰 10개 정보 추가하면 다음 영화로 넘어가기

print(movie_list)
print(title_list)
print(uid_list)
print(url_list)

NameError: ignored

## 리뷰 상세 페이지

In [ ]:
# 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
resp_text = requests.get(url_list[0])
soup_text = BeautifulSoup(resp_text.text, 'html.parser')

# 리뷰 본문의 텍스트를 추출합니다. / <div class = "user_tx_area">
review_text_tag = soup_text.find('div', attrs={'class':'user_tx_area'})

# 텍스트 부분만 추출합니다.
review_text = review_text_tag.get_text()
print(review_text)


2021년이면 내가 18살이구만.. 고2.. 과연그때까지 살아있을까... 제 3세계대전이 일어나서 세계가 멸망할수도 있고 문제인이 나라를 이상하게 만들수도있고... 지금은 그래픽카드가 RTX2080ti 까지나왔다 2021년 에는 RTX9080이 나올수도있을려나 모르겠다 그리고 현재 수학점수 27점 고2에는 제발 100점을 맞게해주세요... 이글은 2년뒤 나한테 쓰는것입니당. 엄마 아빠 사랑해요 2021년 미래인들은 이글을 보고있겠지.........어 시공간에 빨려들어간다 으아아아앙아ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ아아아아아ㅏ응ㄹ엉엌엌ㅋ어ㅏㅋ어카러ㅏ어ㅏ커아컬어ㅏ커아커ㅓㅋ커ㅏ커ㅏ커ㅏㅓ카ㅓㅋㅋ카카커엌ㅇㅋㅇ컹캌아커ㅏ어ㅏ커ㅏ엌ㅇ



## 30개의 리뷰 본문을 모두 수집 -> list로 정리

In [ ]:
# url 반복하여 텍스트를 추출하고 리스트에 추가

text_list = []

for url in url_list:

    # 리뷰 상세 페이지의 HTML 소스코드를 가져와서 파싱(parsing)
    resp_text = requests.get(url)
    soup_text = BeautifulSoup(resp_text.text, 'html.parser')

    # 리뷰 본문의 텍스트를 추출합니다. / <div class = "user_tx_area">
    review_text_tag = soup_text.find('div',attrs={'class':'user_tx_area'})

    # 텍스트 부분만 추출합니다.
    review_text = review_text_tag.get_text()
    text_list.append(review_text)

    # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의)
    time.sleep(2)

# 추출된 아이템의 수량
print(len(text_list))

30


In [ ]:
# url 반복하여 텍스트를 추출하고 리스트에 추가

text_list = []

for url_10 in all_url:
    for url in url_10:

        # 리뷰 상세 페이지의 HTML 소스코드를 가져와서 파싱(parsing)
        resp_text = requests.get(url)
        soup_text = BeautifulSoup(resp_text.text, 'html.parser')

        # 리뷰 본문의 텍스트를 추출합니다. / <div class = "user_tx_area">
        review_text_tag = soup_text.find('div',attrs={'class':'user_tx_area'})

        # 텍스트 부분만 추출합니다.
        review_text = review_text_tag.get_text()
        text_list.append(review_text)

        # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의)
        time.sleep(2)
    # 추출된 아이템의 수량
print(len(text_list))

30


# 데이터 수집

In [ ]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가
dict_data = {
    'movie' : movie_list,
    'title' : title_list,
    'user' : uid_list,
    'review' : text_list
}

# 판다스 데이터프레임으로 변환
df_data = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_data

,movie,title,user,review
0,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,저는 이글을 2년뒤인 2021년에 볼 예정입니다,vita****,\n2021년이면 내가 18살이구만.. 고2.. 과연그때까지 살아있을까... 제 3...
1,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,미래에서 왔습니다,idso****,\n닥터 스트레인지는 타임스톤에 마법을 걸어 두었습니다.타노스에게 넘겨주기 직전 타...
2,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,탈모 스트레인지 2,s220****,\r\n\t\t\t\t이전 닥터 스트레인지 1편을 보신 분들은 아실 겁니다. 마법사...
3,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,(스포 有) 닥스2 캐릭터 아주 자세한 고찰: 닥터 스트레인지: 대혼돈의 멀티버스(...,aasd****,"\n드디어,,마침내,,닥터 스트레인지 2가 개봉했습니다.이 글은 닥터스트레인지2, ..."
4,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,[초대장] 마블 영화 네이버 팬카페,mc_g****,"\n마블 시네마톡 네이버 카페본 카페는 MCU의 세계관을 다룬, 마블 스튜디오가 직..."
5,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,[영화감상] 닥터 스트레인지: 대혼돈의 멀티버스 (Doctor Strange in ...,sakg****,\n5월 4일 개봉하는 영화 '닥터 스트레인지: 대혼돈의 멀티버스'입니다.'마블'시...
6,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,개봉 언제죠... 개봉 하면 당장 달려갑니다...,hmin****,\r\n\t\t\t\t닥터스트레인지1 재밋었는데 2도 엄청 기대중 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ...
7,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,[영화 간단 리뷰] 닥터 스트레인지: 대혼돈의 멀티버스 (2022),choj****,\n오늘은 <닥터 스트레인지: 대혼돈의 멀티버스>를 봤습니다.2016년에 개봉한 <...
8,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,2년뒤에 (인워 스포일러 솔솔(개 소리,char****,\n\r\n(스포일러있음)[세상에.. 일단 타노스가 손가락튕겨서 일단 별점 10...
9,닥터 스트레인지: 대혼돈의 멀티버스 : 네이버 영화,닥터 스트레인지2 대혼돈의 멀티버스 마블영화 쿠키영상 스포일러 후기,ptj0****,\n \n\n\n닥터 스트레인지: 대혼돈의 멀티버스\n\n감독\n샘 레이미\n출연\...


In [ ]:
df_data['movie'] = df_data['movie'].str.split(' : ').str[0]
df_data

,movie,title,user,review
0,닥터 스트레인지: 대혼돈의 멀티버스,저는 이글을 2년뒤인 2021년에 볼 예정입니다,vita****,\n2021년이면 내가 18살이구만.. 고2.. 과연그때까지 살아있을까... 제 3...
1,닥터 스트레인지: 대혼돈의 멀티버스,미래에서 왔습니다,idso****,\n닥터 스트레인지는 타임스톤에 마법을 걸어 두었습니다.타노스에게 넘겨주기 직전 타...
2,닥터 스트레인지: 대혼돈의 멀티버스,탈모 스트레인지 2,s220****,\r\n\t\t\t\t이전 닥터 스트레인지 1편을 보신 분들은 아실 겁니다. 마법사...
3,닥터 스트레인지: 대혼돈의 멀티버스,(스포 有) 닥스2 캐릭터 아주 자세한 고찰: 닥터 스트레인지: 대혼돈의 멀티버스(...,aasd****,"\n드디어,,마침내,,닥터 스트레인지 2가 개봉했습니다.이 글은 닥터스트레인지2, ..."
4,닥터 스트레인지: 대혼돈의 멀티버스,[초대장] 마블 영화 네이버 팬카페,mc_g****,"\n마블 시네마톡 네이버 카페본 카페는 MCU의 세계관을 다룬, 마블 스튜디오가 직..."
5,닥터 스트레인지: 대혼돈의 멀티버스,[영화감상] 닥터 스트레인지: 대혼돈의 멀티버스 (Doctor Strange in ...,sakg****,\n5월 4일 개봉하는 영화 '닥터 스트레인지: 대혼돈의 멀티버스'입니다.'마블'시...
6,닥터 스트레인지: 대혼돈의 멀티버스,개봉 언제죠... 개봉 하면 당장 달려갑니다...,hmin****,\r\n\t\t\t\t닥터스트레인지1 재밋었는데 2도 엄청 기대중 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ...
7,닥터 스트레인지: 대혼돈의 멀티버스,[영화 간단 리뷰] 닥터 스트레인지: 대혼돈의 멀티버스 (2022),choj****,\n오늘은 <닥터 스트레인지: 대혼돈의 멀티버스>를 봤습니다.2016년에 개봉한 <...
8,닥터 스트레인지: 대혼돈의 멀티버스,2년뒤에 (인워 스포일러 솔솔(개 소리,char****,\n\r\n(스포일러있음)[세상에.. 일단 타노스가 손가락튕겨서 일단 별점 10...
9,닥터 스트레인지: 대혼돈의 멀티버스,닥터 스트레인지2 대혼돈의 멀티버스 마블영화 쿠키영상 스포일러 후기,ptj0****,\n \n\n\n닥터 스트레인지: 대혼돈의 멀티버스\n\n감독\n샘 레이미\n출연\...


In [ ]:
# csv 파일로 저장
df_data.to_csv("/content/drive/MyDrive/data/marvel_2022_review.csv")

# 형태소 분석

## 네이버 영화 리뷰 데이터 다운로드

* 출처: https://github.com/e9t/nsmc

In [ ]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok = True)

# 데이터 다운로드
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',path=os.path.join(DATA_DIR, 'review.txt'))

# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv(os.path.join(DATA_DIR, 'review.txt'), sep='\t')

data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## KoNLPy 한글 형태소 분석

In [ ]:
# 긍정 리뷰를 하나 선택 -> label 1
sample_text = data['document'].iloc[102629]
sample_text

# 트위터 형태소 분석기(Okt)를 활용
okt = Okt()

tokens_n = okt.nouns(sample_text) # 명사 추출
pprint(tokens_n)

tokens_v = [x for (x, y) in okt.pos(sample_text) if y == 'Verb'] # 동사 추출
pprint(tokens_v)

['초딩',
 '때',
 '기억',
 '다시',
 '만난',
 '유지인',
 '이덕화',
 '품',
 '대사',
 '영화',
 '제목',
 '집',
 '와이프',
 '이야기',
 '가끔',
 '멘트',
 '목소리',
 '네',
 '해',
 '나',
 '달이']
['봤던', '보내면서', '하는', '하다가', '써', '먹는', '하구요', '되면', '되는구나']


## 텍스트 전처리

* 두 글자 이하 명사, 동사는 분석대상에서 제외

In [ ]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 1500개의 샘플을 선택하여 추출
review_data = review_data[:1500]

# 두 글자 단어 제외
cleaned_review_data = []

for review in tqdm(review_data):
    tokens_n = okt.nouns(review) # 명사
    tokens_v = [x for (x, y) in okt.pos(review) if y == 'Verb'] # 동사
    tokens = tokens_n + tokens_v
    cleaned_tokens = []
    for word in tokens:
        if len(word) >= 3:
            cleaned_tokens.append(word)
        else :
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[757]) # 758번째 데이터 추출

100%|██████████| 1500/1500 [00:10<00:00, 145.40it/s]

1500
다니엘헤니 다니엘헤니 설경구


# 벡터화 표현

In [ ]:
# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[757])

(1500, 1916)
  (0, 972)	0.4472135954999579
  (0, 333)	0.8944271909999159


In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력(앞에서 5개의 단어만 출력)
print({k:v for i, (k,v) in enumerate(vocab.items()) if i < 5})

# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items()}

# 앞에서 5개의 단어를 출력
print({k:v for i, (k,v) in enumerate(index_to_word.items()) if i<5})

1916
{'목소리': 648, '포스터': 1714, '않구나': 1116, '보는것을': 780, '교도소': 93}
{648: '목소리', 1714: '포스터', 1116: '않구나', 780: '보는것을', 93: '교도소'}


In [ ]:
# 758번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[757].indices])
print(original_text)

설경구 다니엘헤니


# 토픽 모델링

## LDA

In [ ]:
# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정 : 긍정 / 부정)
lda1 = LatentDirichletAllocation(n_components = 2)

# LDA 모델링 객체를 생성 (토픽 갯수를 3로 설정)
lda2 = LatentDirichletAllocation(n_components = 3)

# TF-IDF 벡터를 입력하여 모델 학습
print(lda1.fit(review_tfid))
print(lda2.fit(review_tfid))

# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 1916개의 단어)
print(lda1.components_.shape)

# 토픽 모델링 결과를 담고 있는 배열의 형태 : (3개의 토픽, 1916개의 단어)
print(lda2.components_.shape)

LatentDirichletAllocation(n_components=2)
LatentDirichletAllocation(n_components=3)
(2, 1916)
(3, 1916)


In [ ]:
# 1916개의 단어중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력(토픽 2개)
for idx, topic in enumerate(lda1.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('드라마', 23.295), ('쓰레기', 19.063), ('주인공', 12.801), ('시리즈', 8.673), ('한국영', 7.056), ('합니다', 6.761), ('모르겠다', 5.754), ('연기력', 5.629), ('이영화', 5.154), ('시청률', 4.771)]
토픽 유형 2: [('스토리', 21.95), ('봤는데', 12.774), ('이야기', 10.849), ('캐스팅', 8.398), ('마지막', 8.193), ('보면서', 7.344), ('봤어요', 6.706), ('봤습니다', 5.53), ('긴장감', 5.083), ('포스터', 4.931)]


In [ ]:
# 1916개의 단어중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력(토픽 3개)
for idx, topic in enumerate(lda2.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('스토리', 21.411), ('쓰레기', 18.93), ('한국영', 6.917), ('합니다', 6.365), ('포스터', 5.722), ('모르겠다', 5.56), ('시청률', 4.468), ('갈수록', 4.432), ('이야기', 4.397), ('주인공', 4.251)]
토픽 유형 2: [('드라마', 22.53), ('봤는데', 9.278), ('주인공', 8.79), ('시리즈', 8.509), ('캐스팅', 8.284), ('보면서', 7.177), ('이야기', 6.704), ('봤습니다', 5.365), ('만드는', 4.714), ('했는데', 4.656)]
토픽 유형 3: [('봤어요', 6.567), ('마지막', 5.253), ('긴장감', 5.005), ('그대로', 4.395), ('나오고', 4.374), ('최고다', 4.329), ('스릴러', 4.321), ('보는데', 3.961), ('봤는데', 3.682), ('보지마라', 3.596)]


## pyLDAvis 시각화

In [ ]:
# LDA 토픽 모델링 결과를 시각화
# 토픽 2개
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda1, review_tfid, tfid)
pyLDAvis.display(visualization)

In [ ]:
# 토픽 3개
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda2, review_tfid, tfid)
pyLDAvis.display(visualization)

# 감성 분석

## 머신러닝 알고리즘을 활용하여, 긍정/부정 감성을 분류

In [ ]:
labels = data['label'].iloc[:1500].values

# 로지스틱 분류 모델링 객체를 생성
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습
lr.fit(review_tfid, labels)

# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


## 2022 마블 영화 리뷰를 긍정, 부정으로 감성 분류 예측

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/data/marvel_2022_review.csv')

,Unnamed: 0,movie,title,user,review
0,0,닥터 스트레인지: 대혼돈의 멀티버스,저는 이글을 2년뒤인 2021년에 볼 예정입니다,vita****,\n2021년이면 내가 18살이구만.. 고2.. 과연그때까지 살아있을까... 제 3...
1,1,닥터 스트레인지: 대혼돈의 멀티버스,미래에서 왔습니다,idso****,\n닥터 스트레인지는 타임스톤에 마법을 걸어 두었습니다.타노스에게 넘겨주기 직전 타...
2,2,닥터 스트레인지: 대혼돈의 멀티버스,탈모 스트레인지 2,s220****,\r\n\t\t\t\t이전 닥터 스트레인지 1편을 보신 분들은 아실 겁니다. 마법사...
3,3,닥터 스트레인지: 대혼돈의 멀티버스,(스포 有) 닥스2 캐릭터 아주 자세한 고찰: 닥터 스트레인지: 대혼돈의 멀티버스(...,aasd****,"\n드디어,,마침내,,닥터 스트레인지 2가 개봉했습니다.이 글은 닥터스트레인지2, ..."
4,4,닥터 스트레인지: 대혼돈의 멀티버스,[초대장] 마블 영화 네이버 팬카페,mc_g****,"\n마블 시네마톡 네이버 카페본 카페는 MCU의 세계관을 다룬, 마블 스튜디오가 직..."


In [ ]:
test['emotion'] = 0
for i in range(len(test)):
    test_sample = test['review'][i]

    # 세 글자 이상의 명사만을 추출
    tokens_n = okt.nouns(test_sample)
    tokens_v = [x for (x, y) in okt.pos(test_sample) if y == 'Verb'] # 동사
    tokens = tokens_n + tokens_v
    cleaned_tokens = []
    for word in tokens:
        if len(word) >= 3:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)

    # TF-IDF 변환기에 데이터를 입력하여 변환
    test_review_tfid = tfid.transform([cleaned_review])

    # 로지스틱 회귀 모델을 활용하여 분류 예측
    test_pred = lr.predict(test_review_tfid)[0]
    print("분석 결과 {}적인 리뷰로 예측됩니다.".format("긍정" if test_pred > 0 else "부정"))
    test['emotion'].iloc[i] = test_pred

분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.
분석 결과 긍정적인 리뷰로 예측됩니다.
분석 결과 부정적인 리뷰로 예측됩니다.


In [ ]:
test.head()

,Unnamed: 0,movie,title,user,review,emotion
0,0,닥터 스트레인지: 대혼돈의 멀티버스,저는 이글을 2년뒤인 2021년에 볼 예정입니다,vita****,\n2021년이면 내가 18살이구만.. 고2.. 과연그때까지 살아있을까... 제 3...,0
1,1,닥터 스트레인지: 대혼돈의 멀티버스,미래에서 왔습니다,idso****,\n닥터 스트레인지는 타임스톤에 마법을 걸어 두었습니다.타노스에게 넘겨주기 직전 타...,1
2,2,닥터 스트레인지: 대혼돈의 멀티버스,탈모 스트레인지 2,s220****,\r\n\t\t\t\t이전 닥터 스트레인지 1편을 보신 분들은 아실 겁니다. 마법사...,1
3,3,닥터 스트레인지: 대혼돈의 멀티버스,(스포 有) 닥스2 캐릭터 아주 자세한 고찰: 닥터 스트레인지: 대혼돈의 멀티버스(...,aasd****,"\n드디어,,마침내,,닥터 스트레인지 2가 개봉했습니다.이 글은 닥터스트레인지2, ...",0
4,4,닥터 스트레인지: 대혼돈의 멀티버스,[초대장] 마블 영화 네이버 팬카페,mc_g****,"\n마블 시네마톡 네이버 카페본 카페는 MCU의 세계관을 다룬, 마블 스튜디오가 직...",0


In [ ]:
test.groupby('movie')['emotion'].sum()

movie
닥터 스트레인지: 대혼돈의 멀티버스    4
블랙 팬서: 와칸다 포에버         5
토르: 러브 앤 썬더            6
Name: emotion, dtype: int64